# <font color=brown>赛事概要：（结构化：微信大数据挑战赛）</font>

## 本次比赛基于脱敏和采样后的数据信息，对于给定的一定数量到访过微信视频号“热门推荐”的用户， 根据这些用户在视频号内的历史14天的行为数据，通过算法在测试集上预测出这些用户在第15天对于不同视频内容的互动行为（包括点赞、点击头像、收藏、转发等）的发生概率。


In [2]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tqdm import tqdm
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models.deepfm import *
from deepctr_torch.models.basemodel import *
from sklearn.metrics import log_loss, roc_auc_score
#from deepctr.models import DeepFM
from deepctr_torch.models import DeepFM

In [3]:
# 存储数据的根目录
ROOT_PATH = "F:/aliyuun_datas/微信大数据挑战赛"
# 比赛数据集路径
DATASET_PATH = ROOT_PATH + '/wechat_algo_data1/'
# 训练集
USER_ACTION = DATASET_PATH + "user_action.csv"
FEED_INFO = DATASET_PATH + "feed_info.csv"
FEED_EMBEDDINGS = DATASET_PATH + "feed_embeddings.csv"
# 测试集
TEST_FILE = DATASET_PATH + "test_a.csv"

# 初赛待预测行为列表

# ACTION_LIST = 【是否查看评论、是否点赞、是否点击头像、是否转发】
ACTION_LIST = ["read_comment", "like", "click_avatar", "forward"]
# FEA_COLUMN_LIST = 【是否查看评论、是否点赞、是否点击头像、是否转发、是否发表评论、是否关注、是否收藏】
FEA_COLUMN_LIST = ["read_comment", "like", "click_avatar", "forward", "comment", "follow", "favorite"]
# FEA_FEED_LIST =【Feed视频ID、视频号作者ID、Feed时长、背景音乐ID、背景音乐歌手ID】
FEA_FEED_LIST = ['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id']


# 负样本下采样比例(负样本:正样本)
ACTION_SAMPLE_RATE = {"read_comment": 5, "like": 5, "click_avatar": 5, "forward": 10, "comment": 10, "follow": 10,
                      "favorite": 10}

In [6]:
user_action = pd.read_csv(USER_ACTION)
feed_info = pd.read_csv(FEED_INFO)
feed_embddings = pd.read_csv(FEED_EMBEDDINGS)
test_file = pd.read_csv(TEST_FILE)

print(user_action.shape, feed_info.shape, feed_embddings.shape, test_file.shape)

(7317882, 13) (106444, 15) (106444, 2) (421985, 3)


# <font color=orangered>样本处理</font>
### 1. 数据左连接，构建数据集
### 2. 时长 log 化处理
### 3. 去重，保留最后一个
### 4. 采样

结果：
生成1张表（测试集）：
* 1.test_data.csv

生成4张表（训练集）：
* 1.train_data_for_read_comment.csv
* 2.train_data_for_like.csv
* 3.train_data_for_click)avatar.csv
* 4.train_data_for_forward.csv

### 5. 缺失处理

In [6]:
def prepare_data(FEED_INFO, USER_ACTION, FEA_COLUMN_LIST, FEED_EMBEDDINGS, TEST_FILE, FEA_FEED_LIST, ROOT_PATH, ACTION_LIST, ACTION_SAMPLE_RATE):
    feed_info_df = pd.read_csv(FEED_INFO) # 训练集 feed_info.csv
    user_action_df = pd.read_csv(USER_ACTION)[["userid", "date_", "feedid"] + FEA_COLUMN_LIST]
    feed_embed = pd.read_csv(FEED_EMBEDDINGS) # feed_embeddings.csv
    test = pd.read_csv(TEST_FILE) # 测试集 test_a.csv
    # add feed feature
    train = pd.merge(user_action_df, feed_info_df[FEA_FEED_LIST], on='feedid', how='left') # 合并表
    test = pd.merge(test, feed_info_df[FEA_FEED_LIST], on='feedid', how='left') 
    test["videoplayseconds"] = np.log(test["videoplayseconds"] + 1.0) # Feed时长log处理
    test.to_csv(ROOT_PATH + f'/test_data.csv', index=False)
    for action in tqdm(ACTION_LIST):#ACTION_LIST = ["read_comment", "like", "click_avatar", "forward"]
        print(f"prepare data for {action}")
        # 去除重复项，除了最后一次出现的。
        tmp = train.drop_duplicates(['userid', 'feedid', action], keep='last')  
        df_neg = tmp[tmp[action] == 0]#df_neg含义：没有查看评论、没有点赞、没有点击头像、没有转发
        # pd.sample()实现从df中随机抽样,frac参数是抽取行的比例，replace参数是是否有放回的抽样False为无放回
        df_neg = df_neg.sample(frac=1.0 / ACTION_SAMPLE_RATE[action], random_state=42, replace=False)
        df_all = pd.concat([df_neg, tmp[tmp[action] == 1]])
        # Feed时长log处理，避免离散的异常数值影响，同时缓解运算
        df_all["videoplayseconds"] = np.log(df_all["videoplayseconds"] + 1.0) 
        df_all.to_csv(ROOT_PATH + f'/train_data_for_{action}.csv', index=False)


In [7]:
prepare_data(FEED_INFO, USER_ACTION, FEA_COLUMN_LIST, FEED_EMBEDDINGS, TEST_FILE, FEA_FEED_LIST, ROOT_PATH, ACTION_LIST, ACTION_SAMPLE_RATE)

  0%|          | 0/4 [00:00<?, ?it/s]

prepare data for read_comment


 25%|██▌       | 1/4 [00:08<00:25,  8.63s/it]

prepare data for like


 50%|█████     | 2/4 [00:16<00:16,  8.33s/it]

prepare data for click_avatar


 75%|███████▌  | 3/4 [00:24<00:08,  8.08s/it]

prepare data for forward


100%|██████████| 4/4 [00:29<00:00,  7.37s/it]


# <font color=gree>CTR模型：DeepFM
## <font color=bulle>模型采用以下特征：
## <font color=orange>离散特征：'user_id', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id'
## <font color=orange>连续特征：videoplayseconds

In [3]:
class MyBaseModel(BaseModel):

    # 训练
    def fit(self, x=None, y=None, batch_size=None, epochs=1, verbose=1, initial_epoch=0, validation_split=0.,
            validation_data=None, shuffle=True, callbacks=None):

        if isinstance(x, dict):
            x = [x[feature] for feature in self.feature_index]

        do_validation = False
        if validation_data:
            do_validation = True
            if len(validation_data) == 2:
                val_x, val_y = validation_data
                val_sample_weight = None
            elif len(validation_data) == 3:
                val_x, val_y, val_sample_weight = validation_data  # pylint: disable=unpacking-non-sequence
            else:
                raise ValueError(
                    'When passing a `validation_data` argument, '
                    'it must contain either 2 items (x_val, y_val), '
                    'or 3 items (x_val, y_val, val_sample_weights), '
                    'or alternatively it could be a dataset or a '
                    'dataset or a dataset iterator. '
                    'However we received `validation_data=%s`' % validation_data)
            if isinstance(val_x, dict):
                val_x = [val_x[feature] for feature in self.feature_index]

        elif validation_split and 0. < validation_split < 1.:
            do_validation = True
            if hasattr(x[0], 'shape'):
                split_at = int(x[0].shape[0] * (1. - validation_split))
            else:
                split_at = int(len(x[0]) * (1. - validation_split))
            x, val_x = (slice_arrays(x, 0, split_at),
                        slice_arrays(x, split_at))
            y, val_y = (slice_arrays(y, 0, split_at),
                        slice_arrays(y, split_at))
        else:
            val_x = []
            val_y = []
        for i in range(len(x)):
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        train_tensor_data = Data.TensorDataset(
            torch.from_numpy(
                np.concatenate(x, axis=-1)),
            torch.from_numpy(y))
        if batch_size is None:
            batch_size = 256

        model = self.train()
        loss_func = self.loss_func
        optim = self.optim

        if self.gpus:
            print('parallel running on these gpus:', self.gpus)
            model = torch.nn.DataParallel(model, device_ids=self.gpus)
            batch_size *= len(self.gpus)  # input `batch_size` is batch_size per gpu
        else:
            print(self.device)

        train_loader = DataLoader(
            dataset=train_tensor_data, shuffle=shuffle, batch_size=batch_size)

        sample_num = len(train_tensor_data)
        steps_per_epoch = (sample_num - 1) // batch_size + 1

        # configure callbacks
        callbacks = (callbacks or []) + [self.history]  # add history callback
        callbacks = CallbackList(callbacks)
        callbacks.on_train_begin()
        callbacks.set_model(self)
        if not hasattr(callbacks, 'model'):
            callbacks.__setattr__('model', self)
        callbacks.model.stop_training = False

        # Train
        print("Train on {0} samples, validate on {1} samples, {2} steps per epoch".format(
            len(train_tensor_data), len(val_y), steps_per_epoch))
        for epoch in range(initial_epoch, epochs):
            callbacks.on_epoch_begin(epoch)
            epoch_logs = {}
            start_time = time.time()
            loss_epoch = 0
            total_loss_epoch = 0
            train_result = {}
            try:
                with tqdm(enumerate(train_loader), disable=verbose != 1) as t:
                    for _, (x_train, y_train) in t:
                        x = x_train.to(self.device).float()
                        y = y_train.to(self.device).float()

                        y_pred = model(x).squeeze()

                        optim.zero_grad()
                        loss = loss_func(y_pred, y.squeeze(), reduction='sum')
                        reg_loss = self.get_regularization_loss()

                        total_loss = loss + reg_loss + self.aux_loss

                        loss_epoch += loss.item()
                        total_loss_epoch += total_loss.item()
                        total_loss.backward()
                        optim.step()

                        if verbose > 0:
                            for name, metric_fun in self.metrics.items():
                                if name not in train_result:
                                    train_result[name] = []
                                try:
                                    temp = metric_fun(
                                        y.cpu().data.numpy(), y_pred.cpu().data.numpy().astype("float64"))
                                except Exception:
                                    temp = 0
                                finally:
                                    train_result[name].append(temp)
            except KeyboardInterrupt:
                t.close()
                raise
            t.close()

            # Add epoch_logs
            epoch_logs["loss"] = total_loss_epoch / sample_num
            for name, result in train_result.items():
                epoch_logs[name] = np.sum(result) / steps_per_epoch

            if do_validation:
                eval_result = self.evaluate(val_x, val_y, batch_size)
                for name, result in eval_result.items():
                    epoch_logs["val_" + name] = result
            # verbose
            if verbose > 0:
                epoch_time = int(time.time() - start_time)
                print('Epoch {0}/{1}'.format(epoch + 1, epochs))

                eval_str = "{0}s - loss: {1: .4f}".format(
                    epoch_time, epoch_logs["loss"])

                for name in self.metrics:
                    eval_str += " - " + name + \
                                ": {0: .4f}".format(epoch_logs[name])

                if do_validation:
                    for name in self.metrics:
                        eval_str += " - " + "val_" + name + \
                                    ": {0: .4f}".format(epoch_logs["val_" + name])
                print(eval_str)
            callbacks.on_epoch_end(epoch, epoch_logs)
            if self.stop_training:
                break

        callbacks.on_train_end()

        return self.history

    # 评估
    def evaluate(self, x, y, batch_size=256):
        """
        ：param x:Numpy测试数据数组（如果模型有一个输入），或Numpy数组列表（如果模型有多个输入）。
        ：param y:Numpy目标（标签）数据数组（如果模型有一个输出），或Numpy数组列表（如果模型有多个输出）。
        ：param batch_size:整数或“无”。每个评估步骤的样本数。如果未指定，“批次大小”将默认为256。
        ：return:Dict包含度量名称和度量值。
        """
        pred_ans = self.predict(x, batch_size)
        eval_result = {}
        for name, metric_fun in self.metrics.items():
            try:
                temp = metric_fun(y, pred_ans)
            except Exception:
                temp = 0
            finally:
                eval_result[name] = metric_fun(y, pred_ans)
        return eval_result

    # 预测
    def predict(self, x, batch_size=256):
        """
        ：param x：输入数据，作为Numpy数组（如果模型有多个输入，则为Numpy数组列表）。
        ：param batch_size:整数。如果未指定，则默认为256。
        ：return:Numpy预测数组。
        """
        model = self.eval()
        if isinstance(x, dict):
            x = [x[feature] for feature in self.feature_index]
        for i in range(len(x)):
            if len(x[i].shape) == 1:
                x[i] = np.expand_dims(x[i], axis=1)

        tensor_data = Data.TensorDataset(
            torch.from_numpy(np.concatenate(x, axis=-1)))
        test_loader = DataLoader(
            dataset=tensor_data, shuffle=False, batch_size=batch_size)

        pred_ans = []
        with torch.no_grad():
            for _, x_test in enumerate(test_loader):
                x = x_test[0].to(self.device).float()

                y_pred = model(x).cpu().data.numpy()  # .squeeze()
                pred_ans.append(y_pred)

        return np.concatenate(pred_ans).astype("float64")

In [4]:
class MyDeepFM(MyBaseModel):
    def __init__(self,
                 linear_feature_columns, dnn_feature_columns, use_fm=True,
                 dnn_hidden_units=(256, 128),
                 l2_reg_linear=0.00001, l2_reg_embedding=0.00001, l2_reg_dnn=0, init_std=0.0001, seed=1024,
                 dnn_dropout=0,
                 dnn_activation='relu', dnn_use_bn=False, task='binary', device='cpu', gpus=None):

        # 调用父类构造函数，初始化模型参数
        super(MyDeepFM, self).__init__(linear_feature_columns, dnn_feature_columns, l2_reg_linear=l2_reg_linear,
                                     l2_reg_embedding=l2_reg_embedding, init_std=init_std, seed=seed, task=task,
                                     device=device, gpus=gpus)

        self.use_fm = use_fm
        self.use_dnn = len(dnn_feature_columns) > 0 and len(
            dnn_hidden_units) > 0
        if use_fm:
            self.fm = FM()

        if self.use_dnn:
            self.dnn = DNN(self.compute_input_dim(dnn_feature_columns), dnn_hidden_units,
                           activation=dnn_activation, l2_reg=l2_reg_dnn, dropout_rate=dnn_dropout, use_bn=dnn_use_bn,
                           init_std=init_std, device=device)
            self.dnn_linear = nn.Linear(
                dnn_hidden_units[-1], 1, bias=False).to(device)

            self.add_regularization_weight(
                filter(lambda x: 'weight' in x[0] and 'bn' not in x[0], self.dnn.named_parameters()), l2=l2_reg_dnn)
            self.add_regularization_weight(self.dnn_linear.weight, l2=l2_reg_dnn)
        self.to(device)

    def forward(self, X):

        sparse_embedding_list, dense_value_list = self.input_from_feature_columns(X, self.dnn_feature_columns,
                                                                                  self.embedding_dict)
        logit = self.linear_model(X)

        if self.use_fm and len(sparse_embedding_list) > 0:
            fm_input = torch.cat(sparse_embedding_list, dim=1)
            logit += self.fm(fm_input)

        if self.use_dnn:
            dnn_input = combined_dnn_input(
                sparse_embedding_list, dense_value_list)
            dnn_output = self.dnn(dnn_input)
            dnn_logit = self.dnn_linear(dnn_output)
            logit += dnn_logit

        y_pred = self.out(logit)

        return y_pred

# <font color=yellow>待预测行为列表</font>
### <font color=orange>ACTION_LIST = 【是否查看评论、是否点赞、是否点击头像、是否转发】
### ACTION_LIST = ["read_comment", "like", "click_avatar", "forward"]
### FEA_COLUMN_LIST = 【是否查看评论、是否点赞、是否点击头像、是否转发、是否发表评论、是否关注、是否收藏】
### FEA_COLUMN_LIST = ["read_comment", "like", "click_avatar", "forward", "comment", "follow", "favorite"]
### EA_FEED_LIST =【Feed视频ID、视频号作者ID、Feed时长、背景音乐ID、背景音乐歌手ID】
### FEA_FEED_LIST = ['feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id']


* <font color=grep>FM：
    * 相当于对逻辑回归做了改进，加了特征交叉；目的是解决数据稀疏的情况下，特征怎样组合的问题。
    * 主要应用场景是点击率预估，目的是在数据高维稀疏的情况下，解决特征的组合问题。
*

* DNN基本结构：
    * 感知机：若干输入和一个输出，输入和输出是线性关系，还有一个神经元激活函数，此模型只能用于二分类问题。
    * 神经网络：（1）加入隐藏层，增强模型表达能力。（2）多个输出。（3）激活函数的扩展。
    * 神经网络是基于感知机的扩展，DNN可以理解为有很多隐藏层的神经网络。多层神经网络和DNN其实也指的一个东西，DNN有时也叫多层感知机。
* 

* <font color=grep>DeepFM：将深度神经网络模型与FM模型结合，同时学习低阶和高阶的特征交叉，主要由FM和DNN两部分组成，底部共享同样的输入。FM模型善于挖掘二阶特征交叉关系，而神经网络DNN的优点是能够挖掘高阶的特征交叉关系，于是DeepFM将两者组合到一起，实验证明DeepFM比单模型FM、DNN效果好。


In [5]:
if __name__ == "__main__":
    submit = pd.read_csv(ROOT_PATH + '/test_data.csv')[['userid', 'feedid']]
    for action in ACTION_LIST:  # 预测行为列表["read_comment", "like", "click_avatar", "forward"]
        USE_FEAT = ['userid', 'feedid', action] + FEA_FEED_LIST[1:]
        train = pd.read_csv(ROOT_PATH + f'/train_data_for_{action}.csv')[USE_FEAT]
        train = train.sample(frac=1, random_state=42).reset_index(drop=True)
        print("posi prop:")  
        print(sum((train[action]==1)*1)/train.shape[0]) # 1值占比情况
        test = pd.read_csv(ROOT_PATH + '/test_data.csv')[[i for i in USE_FEAT if i != action]]
        # [['userid', 'feedid', 'authorid', 'videoplayseconds', 'bgm_song_id', 'bgm_singer_id']]

        target = [action]
        test[target[0]] = 0
        test = test[USE_FEAT]
        data = pd.concat((train, test)).reset_index(drop=True)
        dense_features = ['videoplayseconds']
        sparse_features = [i for i in USE_FEAT if i not in dense_features and i not in target]
        # 稀疏特征sparse_features = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id']

        data[sparse_features] = data[sparse_features].fillna(0) # 缺失值处理
        data[dense_features] = data[dense_features].fillna(0)

        # 1.对稀疏特征进行标签编码，对稠密特征进行0-1标准化缩放。稀疏：大部分都是0值
        for feat in sparse_features:
            lbe = LabelEncoder()
            data[feat] = lbe.fit_transform(data[feat])
        mms = MinMaxScaler(feature_range=(0, 1))
        data[dense_features] = mms.fit_transform(data[dense_features])

        # 2.计算每个稀疏特征的不同值个数，并记录字段名
        # 这里是比较关键的一步，因为需要对类别特征进行Embedding，
        # 所以需要告诉模型一个特征组有多少个embbedding向量。（nunique）
        fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                                  for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                                  for feat in dense_features]
        dnn_feature_columns = fixlen_feature_columns
        linear_feature_columns = fixlen_feature_columns

        feature_names = get_feature_names(
            linear_feature_columns + dnn_feature_columns)
        # feature_names = ['userid', 'feedid', 'authorid', 'bgm_song_id', 'bgm_singer_id', 'videoplayseconds']

        # 3.为模型生成输入数据
        train, test = data.iloc[:train.shape[0]].reset_index(drop=True), data.iloc[train.shape[0]:].reset_index(drop=True)
        train_model_input = {name: train[name] for name in feature_names}
        test_model_input = {name: test[name] for name in feature_names}

        # 4.定义模型，训练，预测，评估
        # 检查是否可以使用GPU
        device = 'cpu'
        use_cuda = True 
        if use_cuda and torch.cuda.is_available():
            print('cuda ready...')
            device = 'cuda:0'

        # 初始化模型
        model = MyDeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                       task='binary',
                       l2_reg_embedding=1e-1, device=device)

        model.compile("adagrad", "binary_crossentropy", metrics=["binary_crossentropy", "auc"])

        history = model.fit(train_model_input, train[target].values, 
                            batch_size=512, # 定义每次训练的批量数（整数型），默认为32 
                            epochs=5,   # 训练模型的次数
                            verbose=1,  # 日志显示，1表示输出进度条记录
                            validation_split=0.2    # 划分验证集，数据集中序列靠后的数据进行划分
                            )
        pred_ans = model.predict(test_model_input, 128)
        submit[action] = pred_ans
        torch.cuda.empty_cache()
    # 保存提交文件
    # submit.to_csv("./submit_base_deepfm.csv", index=False)

posi prop:
0.15565121499983292


2it [00:00, 19.66it/s]

cpu
Train on 1316708 samples, validate on 329177 samples, 2572 steps per epoch


2572it [01:17, 33.00it/s]
2it [00:00, 17.00it/s]

Epoch 1/5
79s - loss:  0.2507 - binary_crossentropy:  0.2351 - auc:  0.9227 - val_binary_crossentropy:  0.2167 - val_auc:  0.9392


...

Epoch 3/5
32s - loss:  0.0982 - binary_crossentropy:  0.0881 - auc:  0.9496 - val_binary_crossentropy:  0.1092 - val_auc:  0.8998


1165it [00:31, 36.43it/s]
3it [00:00, 28.38it/s]

Epoch 4/5
32s - loss:  0.0922 - binary_crossentropy:  0.0829 - auc:  0.9582 - val_binary_crossentropy:  0.1128 - val_auc:  0.8969


1165it [00:31, 36.46it/s]


Epoch 5/5
32s - loss:  0.0880 - binary_crossentropy:  0.0794 - auc:  0.9633 - val_binary_crossentropy:  0.1159 - val_auc:  0.8949
